In [1]:
import requests as rq
import pandas as pd
from datetime import datetime as dt, timedelta as td
import json as js

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 150)

In [ ]:
url = "https://raw.githubusercontent.com/tofrie03/ML_Assignment/refs/heads/main/Material/Energydata_Spain_2020.csv"

df = pd.read_csv(url)

print(df)

def clean_data(df):
    try:
        # Ensures that the datetime column is in datetime-format
        df["datetime"] = pd.to_datetime(df["datetime"])

        print(df)
        print(df["datetime"].dtype)

        





        # # Counts the number of na values in each column
        # value_na, percentage_na, datetime_na = pd.isna(df).sum()
        # if value_na != 0:
        #     df["value"].fillna(df["value"].mean(), inplace=True)
        #     print(f"Filled NA values in 'value' column with mean: {df['value'].mean()}")
        # if percentage_na != 0:
        #     df["percentage"].fillna(df["percentage"].mean(), inplace=True)
        #     print(f"Filled NA values in 'percentage' column with mean: {df['percentage'].mean()}")
        # total_na = pd.isna(df).sum().sum()
        # print(total_na, total_na.dtype)
        # if total_na != 0:
        #     print(f"Total NA values in the dataframe: {total_na}")
        # else:
        #     print("No NA values found in the dataframe.")
    except Exception as e:
        return e
        

clean_data(df)

print(df)

      value  percentage             datetime  Wind  Nuclear  Coal  Combined cycle  Int. exchanges  Solar PV  Solar thermal  Thermal renewable  \
0     38.60           1  2020-01-01 00:00:00  1597     7097   434            5051             301        17              1                362   
1     36.55           1  2020-01-01 01:00:00  1662     7100   435            5416             249        17              1                391   
2     32.32           1  2020-01-01 02:00:00  1733     7098   428            5189            -636        17              1                385   
3     30.85           1  2020-01-01 03:00:00  1727     7098   409            4723           -1109        17              1                378   
4     30.14           1  2020-01-01 04:00:00  1671     7098   390            4753           -1204        17              1                377   
...     ...         ...                  ...   ...      ...   ...             ...             ...       ...            ...        